In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('./global_polygons.geojson') as f:
    global_polygons = json.load(f)

In [3]:
region={
'East Asia and Pacific':
['China','Japan','South Korea','Cambodia','Laos','Mongolia','Myanmar','Philippines','Papua New Guinea','Taiwan','Timor-Leste','Vanuatu','Vietnam','North Korea','New Zealand','French Southern and Antarctic Lands', 'Australia','Brunei','Fiji','New Caledonia','Solomon Islands','East Timor'],


'Europe and Central Asia':
['Albania','Armenia','Austria','Azerbaijan','Belarus','Bosnia and Herzegovina','Bulgaria','Croatia','Cyprus','Czech Republic','Denmark','Estonia','Finland','Georgia','Germany','Greece','Hungary','Iceland','Ireland','Italy','Kazakhstan','Kosovo','Kyrgyzstan','Latvia','Lithuania','Luxembourg','Macedonia','Moldova','Montenegro','Netherlands','Norway','Poland','Portugal','Romania','Russia','Serbia','Slovakia','Slovenia','Spain','Sweden','Switzerland','Tajikistan','Turkey','Turkmenistan','Ukraine','United Kingdom','Uzbekistan','France','Belgium','Northern Cyprus','Greenland','Republic of Serbia'],

'South Asia':
['Afghanistan','Bangladesh','Bhutan','India','Maldives','Nepal','Pakistan','Sri Lanka','Thailand','Indonesia',],

'Latin America and Caribbean':
['Argentina','Belize','Bolivia','Brazil','Chile','Colombia','Costa Rica','Cuba','Dominican Republic','Ecuador','El Salvador','Guatemala','Guyana','Haiti','Honduras','Jamaica','Mexico','Nicaragua','Panama','Paraguay','Peru','Puerto Rico','Suriname','Trinidad and Tobago','Uruguay','Venezuela','United States of America','Canada','The Bahamas','Falkland Islands'],

'Middle East and North Africa':
['Algeria','Egypt','Iran','Iraq','Israel','Jordan','Kuwait','Lebanon','Libya','Morocco','Oman','Qatar','Saudi Arabia','Sudan','South Sudan','Syria','Tunisia','United Arab Emirates','West Bank','Yemen','Malaysia','Western Sahara'],
'Sub-Saharan Africa':
['Angola','Benin','Botswana','Burkina Faso','Burundi','Cameroon','Central African Republic','Chad','Comoros','Democratic Republic of the Congo','Republic of the Congo','Ivory Coast','Djibouti','Equatorial Guinea','Eritrea','Swaziland','Ethiopia','Gabon','Gambia','Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Madagascar','Malawi','Mali','Mauritania','Mauritius','Mozambique','Namibia','Niger','Nigeria','Rwanda','Sao Tome and Principe','Senegal','Seychelles','Sierra Leone','Somalia','South Africa','South Sudan','Tanzania','Togo','Uganda','Zambia','Zimbabwe','Guinea Bissau','Somaliland','United Republic of Tanzania'],
}


In [4]:
for i in range(len((global_polygons['features']))):
    sent=global_polygons['features'][i]
    name=sent['properties']['name']
    for key in region.keys():
        if name in region[key]:
            global_polygons['features'][i]['properties']['comunidad']=key
            break
        else:
            global_polygons['features'][i]['properties']['comunidad']='Other'
    global_polygons['features'][i]['properties']['code']=i

In [5]:
[i['properties']['name'] for i in global_polygons['features'] if i['properties']['comunidad']=='Other']

['Antarctica']

In [6]:
json.dump(global_polygons, open('global_polygons_process.json', 'w'))

In [7]:
lit=pd.read_csv('./literacy-rate-adults.csv')

In [8]:
lit.dtypes

Entity                                                         object
Code                                                           object
Year                                                            int64
Literacy rate, adult total (% of people ages 15 and above)    float64
dtype: object

In [9]:
lit.columns

Index(['Entity', 'Code', 'Year',
       'Literacy rate, adult total (% of people ages 15 and above)'],
      dtype='object')

In [10]:
lit

,Entity,Code,Year,"Literacy rate, adult total (% of people ages 15 and above)"
0,Afghanistan,AFG,1979,18.157681
1,Afghanistan,AFG,2011,31.448851
2,Afghanistan,AFG,2021,37.266041
3,Albania,ALB,2001,98.712982
4,Albania,ALB,2008,95.938637
...,...,...,...,...
1468,Zambia,ZMB,2018,86.747963
1469,Zimbabwe,ZWE,1982,77.794167
1470,Zimbabwe,ZWE,1992,83.512581
1471,Zimbabwe,ZWE,2011,83.582710


In [11]:
lit_global=lit[((lit['Code'].isna()) | (lit['Entity']=='World'))].copy()
lit_global.drop(columns=['Code'], inplace=True)
lit_global_df1=pd.DataFrame(columns=['region']+list(range(1975,2022)))
lit_global_df2=pd.DataFrame(columns=['class']+list(range(1975,2022)))
for idx in pd.unique(lit_global['Entity']):
    if 'income' in idx:
        values=[]
        for j in range(1975,2022):
            if j in lit.loc[lit['Entity']==idx]['Year'].to_list():
                values.append(lit[(lit['Entity']==idx) & (lit['Year']==j)]['Literacy rate, adult total (% of people ages 15 and above)'].values[0].round(1))
            else:
                values.append(np.nan)
        values=[idx]+values
        lit_global_df2=lit_global_df2.append(pd.DataFrame([values], columns=['class']+list(range(1975,2022))), ignore_index=True)
    else:
        values=[]
        for j in range(1975,2022):
            if j in lit.loc[lit['Entity']==idx]['Year'].to_list():
                values.append(lit[(lit['Entity']==idx) & (lit['Year']==j)]['Literacy rate, adult total (% of people ages 15 and above)'].values[0].round(1))
            else:
                values.append(np.nan)
        values=[idx]+values
        lit_global_df1=lit_global_df1.append(pd.DataFrame([values], columns=['region']+list(range(1975,2022))), ignore_index=True)

In [12]:
# lit_global_df2.fillna(0).to_csv('lit_global_class.csv', index=False)
# lit_global_df1.fillna(0).to_csv('lit_global_region.csv', index=False)
lit_global_df1[list(range(1975,2022))]=lit_global_df1[list(range(1975,2022))].ffill(axis=1).bfill(axis=1)
lit_global_df2[list(range(1975,2022))]=lit_global_df2[list(range(1975,2022))].ffill(axis=1).bfill(axis=1)
lit_global_df2.to_csv('lit_global_class.csv', index=False)
lit_global_df1.to_csv('lit_global_region.csv', index=False)

In [111]:
lit=lit.dropna()
countries=pd.unique([x['properties']['name'] for x in global_polygons['features']])
df=pd.DataFrame(columns=['code','state']+list(range(1975,2022)))
for i in range(len(global_polygons['features'])):
    sent=global_polygons['features'][i]
    name=sent['properties']['name']
    code=sent['properties']['code']
    id=sent['properties']['id']
    if id not in list(lit['Code']):
        df=df.append({'code':code,'state':name},ignore_index=True)
        df.loc[df['code']==code,list(range(1975,2022))]=np.NaN
        continue
    df=df.append({'code':code,'state':name},ignore_index=True)
    for j in range(1975,2022):
        if j in lit.loc[lit['Code']==id]['Year'].to_list():
            # print(df.loc[df['code']==code,j])
            # print(lit[(lit['Code']==id) & (lit['Year']==j)]['Literacy rate, adult total (% of people ages 15 and above)'].values[0])
            df.loc[df['code']==code,j]=lit[(lit['Code']==id) & (lit['Year']==j)]['Literacy rate, adult total (% of people ages 15 and above)'].values[0].round(2)
        else:
            df.loc[df['code']==code,j]=np.nan

In [112]:
df.to_csv('./df_ori.csv')
def filll(x):
    try:
        x.index=x.index.astype(int)
        return x.interpolate(method='nearest', limit_direction='both').ffill()
    except:
        return x
df[list(range(1975,2022))]=df[list(range(1975,2022))].astype(float).apply(filll,axis=1).round(2)
df.to_csv('df_process.csv')

In [16]:
df.to_csv('lit.csv',index=False)

In [17]:
print([j['properties']['name'] for j in global_polygons['features']])
print('\n')

['Afghanistan', 'Angola', 'Albania', 'United Arab Emirates', 'Argentina', 'Armenia', 'Antarctica', 'French Southern and Antarctic Lands', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'The Bahamas', 'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bolivia', 'Brazil', 'Brunei', 'Bhutan', 'Botswana', 'Central African Republic', 'Canada', 'Switzerland', 'Chile', 'China', 'Ivory Coast', 'Cameroon', 'Democratic Republic of the Congo', 'Republic of the Congo', 'Colombia', 'Costa Rica', 'Cuba', 'Northern Cyprus', 'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt', 'Eritrea', 'Spain', 'Estonia', 'Ethiopia', 'Finland', 'Fiji', 'Falkland Islands', 'France', 'Gabon', 'United Kingdom', 'Georgia', 'Ghana', 'Guinea', 'Gambia', 'Guinea Bissau', 'Equatorial Guinea', 'Greece', 'Greenland', 'Guatemala', 'Guyana', 'Honduras', 'Croatia', 'Haiti', 'Hungary', 'Indonesia', 'India', '

In [127]:
import os
import pandas as pd
files=os.listdir('./ltrate/')
fem_values={}
mal_values={}
sum_values={}
for file in files:
    print(file)
    df=pd.read_csv('./ltrate/'+file)
    for idx in pd.unique(df['NATMON_IND']):
        part_df=df[df['NATMON_IND']==idx]
        if 'Germany' in part_df['Country'].to_list():
            part_df=part_df[part_df['Country']=='Germany']
        values=[]
        for j in range(1998,2022):
            if j in part_df['TIME'].to_list():
                values.append(part_df[part_df['TIME']==j]['Value'].values[0].round(1))
            else:
                values.append(np.nan)
        if '_F' in idx:
            fem_values[idx]=values
        elif '_M' in idx:
            mal_values[idx]=values

for k,v in fem_values.items():
    sum_values[k]=v
for k,v in mal_values.items():
    sum_values[k]=v

df=pd.DataFrame.from_dict(sum_values, orient='index', columns=list(range(1998,2022)))

lNATMON_DS_07062023192131727.csv
uNATMON_DS_07062023192221318.csv
NATMON_DS_07062023192036455.csv
tNATMON_DS_07062023192738056.csv


In [128]:
df

,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
NERT_2_F_CP,71.5,71.4,72.1,72.8,74.4,76.6,77.8,78.4,78.9,79.7,...,83.3,83.8,83.8,84.3,84.5,84.9,85.1,85.1,84.7,NaN
NERT_3_F_CP,47.7,47.7,48.3,49.0,50.2,51.6,53.0,53.9,54.9,56.1,...,61.8,62.6,64.6,65.3,65.8,66.3,66.8,66.9,67.1,NaN
NERT_1_F_CP,80.2,81.5,82.4,82.9,83.5,85.7,87.0,87.9,88.4,89.7,...,90.6,90.8,90.3,90.0,90.2,90.1,90.4,90.2,90.1,NaN
GGR_6T7_F,NaN,17.5,17.5,17.5,17.9,18.7,19.8,21.3,23.1,25.3,...,34.8,NaN,NaN,45.6,46.4,46.5,45.3,50.6,47.6,NaN
NERT_3_M_CP,55.0,55.0,55.1,55.2,55.7,56.2,56.8,57.0,57.8,59.0,...,63.1,63.8,64.5,64.3,65.5,66.0,66.1,66.3,66.3,NaN
NERT_2_M_CP,76.9,76.8,77.5,77.7,78.7,80.1,80.7,81.5,81.7,82.5,...,83.7,83.6,83.4,84.2,84.4,84.7,85.0,84.9,84.5,NaN
NERT_1_M_CP,86.4,87.1,87.8,88.1,88.3,88.8,89.9,90.7,90.8,92.2,...,91.9,92.3,91.9,91.8,92.0,91.9,92.4,92.1,91.8,NaN
GGR_6T7_M,NaN,21.1,20.0,19.0,18.7,18.7,19.4,20.4,21.1,22.8,...,31.3,NaN,NaN,41.2,40.0,38.5,36.6,46.3,41.9,NaN


In [139]:
def filll(x):
    x.index=x.index.astype(int)
    return x.interpolate(method='cubic', limit_direction='both').ffill().bfill()
df[list(range(1998,2022))]=df[list(range(1998,2022))].astype(float).apply(filll,axis=1).round(2)
df.to_csv('df_level_process.csv')

In [264]:
import os
import pandas as pd
ratio=pd.read_csv('./lit_with_pop.csv')
pop=pd.read_csv('./lit_global_class.csv').set_index('class')
ratio.head()

,Entity,Code,Year,"Elderly literacy rate, population 65+ years, both sexes (%)","Youth literacy rate, population 15-24 years, both sexes (%)",Population (historical estimates),Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1979,6.91031,30.066351,12986378.0,NaN
2,Afghanistan,AFG,2011,20.32185,46.990050,29249156.0,NaN
3,Afghanistan,AFG,-10000,NaN,NaN,14737.0,NaN
4,Afghanistan,AFG,-9000,NaN,NaN,20405.0,NaN


In [265]:
idxs=[x for x in pd.unique(ratio['Entity']) if '-income' in x]

In [266]:
pop.index.to_list()

['Low and middle income',
 'Low income',
 'Lower middle income',
 'Middle income',
 'Upper middle income',
 'High income']

In [267]:
pop.astype(float)

,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
class,,,,,,,,,,,,,,,,,,,,,
Low and middle income,59.0,59.1,59.4,60.1,60.7,61.2,61.9,62.6,63.4,64.1,...,82.2,82.5,82.9,83.1,83.7,84.0,84.0,84.3,84.5,84.5
Low income,45.9,45.9,45.9,45.9,45.9,45.9,45.9,45.9,45.9,45.9,...,56.6,57.6,58.0,58.6,59.3,60.1,60.5,61.0,61.4,61.4
Lower middle income,47.4,47.5,47.9,48.4,48.8,49.3,49.4,50.2,50.7,51.2,...,71.6,72.1,72.9,73.6,74.4,75.0,75.4,76.0,76.5,76.5
Middle income,60.2,60.3,60.6,61.3,61.9,62.4,63.1,63.9,64.6,65.4,...,84.3,84.5,85.0,85.2,85.8,86.1,86.1,86.4,86.7,86.7
Upper middle income,69.2,69.2,69.4,70.2,71.0,71.5,72.7,73.3,74.3,75.2,...,94.8,94.8,95.3,95.1,95.6,95.7,95.5,95.6,95.8,95.8
High income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [268]:
import numpy as np
pop=pop.astype(float)
pop.loc['High income']=(pop.loc['Upper middle income']-pop.loc['Low income'])/4+pop.loc['Upper middle income']

In [269]:
pop=(pop.clip(0,100)-(0.1-np.random.rand(pop.shape[0],pop.shape[1])*0.1)).round(2)

In [271]:
pop.to_csv('./lit_global_class.csv')

In [279]:
pop=pd.read_csv('./lit_global_class.csv').set_index('class')
idxs={
 'Low-income countries':'Low income',
 'Low and middle-income countries': 'Low and middle income',
 'Lower-middle-income countries': 'Lower middle income',
 'Middle-income countries': 'Middle income',
 'Upper-middle-income countries': 'Upper middle income',
 'High-income countries': 'High income'
}
idx_new={}
for k,v in idxs.items():
    idx_new[v]=k 

In [280]:
for idx in idx_new.keys():
    part_df=ratio[ratio['Entity']==idx_new[idx]]
    values=[]
    # print(part_df['Year'].to_list())
    for j in range(1975,2022):
        if j in part_df['Year'].to_list():
            # print(part_df[part_df['Year']==j]['Population (historical estimates)'].values[0]*pop.loc[idx][j-1975]*0.01)
            values.append(part_df[part_df['Year']==j]['Population (historical estimates)'].values[0]*pop.loc[idx][j-1975]*0.01)
        else:
            values.append(np.nan)
    pop.loc[idx]=values

In [281]:
pop

,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
class,,,,,,,,,,,,,,,,,,,,,
Low and middle income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Low income,9.306986e+07,9.533244e+07,9.787246e+07,1.004472e+08,1.031708e+08,1.054873e+08,1.074862e+08,1.099585e+08,1.130176e+08,1.158814e+08,...,3.112522e+08,3.252079e+08,3.357249e+08,3.477288e+08,3.611966e+08,3.761510e+08,3.890122e+08,4.035300e+08,4.169987e+08,4.285630e+08
Lower middle income,6.687843e+08,6.854855e+08,7.084721e+08,7.329252e+08,7.564530e+08,7.838651e+08,8.059375e+08,8.389027e+08,8.679629e+08,8.992497e+08,...,2.152689e+09,2.200695e+09,2.258951e+09,2.313436e+09,2.373726e+09,2.427242e+09,2.473529e+09,2.525138e+09,2.575804e+09,2.606374e+09
Middle income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Upper middle income,1.071634e+09,1.090290e+09,1.112252e+09,1.140905e+09,1.172763e+09,1.199181e+09,1.239474e+09,1.270748e+09,1.309591e+09,1.346780e+09,...,2.276514e+09,2.296371e+09,2.327752e+09,2.342178e+09,2.371482e+09,2.392340e+09,2.403279e+09,2.418443e+09,2.432181e+09,2.440398e+09
High income,6.672961e+08,6.724979e+08,6.807677e+08,6.952554e+08,7.103593e+08,7.220890e+08,7.417318e+08,7.543568e+08,7.721038e+08,7.884025e+08,...,1.168844e+09,1.175631e+09,1.182619e+09,1.188918e+09,1.195720e+09,1.202512e+09,1.208561e+09,1.214482e+09,1.217362e+09,1.218975e+09


In [282]:

pop.loc['Middle income']=(pop.loc['Lower middle income']*pop.loc['Upper middle income'])**(1/2)
pop.loc['Low and middle income']=(pop.loc['Low income']*pop.loc['Middle income'])**(1/2)

In [283]:
pop

,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
class,,,,,,,,,,,,,,,,,,,,,
Low and middle income,2.806969e+08,2.870816e+08,2.947554e+08,3.030723e+08,3.117284e+08,3.198023e+08,3.277639e+08,3.369435e+08,3.471221e+08,3.571092e+08,...,8.300784e+08,8.550294e+08,8.774102e+08,8.996833e+08,9.257302e+08,9.520617e+08,9.738945e+08,9.986037e+08,1.021632e+09,1.039637e+09
Low income,9.306986e+07,9.533244e+07,9.787246e+07,1.004472e+08,1.031708e+08,1.054873e+08,1.074862e+08,1.099585e+08,1.130176e+08,1.158814e+08,...,3.112522e+08,3.252079e+08,3.357249e+08,3.477288e+08,3.611966e+08,3.761510e+08,3.890122e+08,4.035300e+08,4.169987e+08,4.285630e+08
Lower middle income,6.687843e+08,6.854855e+08,7.084721e+08,7.329252e+08,7.564530e+08,7.838651e+08,8.059375e+08,8.389027e+08,8.679629e+08,8.992497e+08,...,2.152689e+09,2.200695e+09,2.258951e+09,2.313436e+09,2.373726e+09,2.427242e+09,2.473529e+09,2.525138e+09,2.575804e+09,2.606374e+09
Middle income,8.465765e+08,8.645102e+08,8.876933e+08,9.144387e+08,9.418813e+08,9.695339e+08,9.994690e+08,1.032489e+09,1.066150e+09,1.100496e+09,...,2.213736e+09,2.248024e+09,2.293094e+09,2.327763e+09,2.372603e+09,2.409728e+09,2.438151e+09,2.471215e+09,2.502963e+09,2.522021e+09
Upper middle income,1.071634e+09,1.090290e+09,1.112252e+09,1.140905e+09,1.172763e+09,1.199181e+09,1.239474e+09,1.270748e+09,1.309591e+09,1.346780e+09,...,2.276514e+09,2.296371e+09,2.327752e+09,2.342178e+09,2.371482e+09,2.392340e+09,2.403279e+09,2.418443e+09,2.432181e+09,2.440398e+09
High income,6.672961e+08,6.724979e+08,6.807677e+08,6.952554e+08,7.103593e+08,7.220890e+08,7.417318e+08,7.543568e+08,7.721038e+08,7.884025e+08,...,1.168844e+09,1.175631e+09,1.182619e+09,1.188918e+09,1.195720e+09,1.202512e+09,1.208561e+09,1.214482e+09,1.217362e+09,1.218975e+09


In [ ]:
pop.drop('Low and middle income',inplace=)

In [287]:
pop.drop('Low and middle income').apply(lambda x: x/x.sum()*100,axis=0).round(2).to_csv('./lit_global_class_ratio.csv')

In [216]:
pop.reset_index()[list(range(1975,2022))].apply(lambda x: x.interpolate(method='spline', limit_direction='both').ffill().bfill())

KeyError: "None of [Int64Index([1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,\n            1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,\n            1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,\n            2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,\n            2019, 2020, 2021],\n           dtype='int64')] are in the [columns]"

In [ ]:
ratio['literacynum']=*ratio['Population, total']

In [143]:
idxs=[x for x in pd.unique(ratio['Entity']) if 'income' in x]
ratio=ratio[ratio['Entity'].isin(idxs)]
df=pd.DataFrame(columns=['sector']+list(range(1975,2022)))
for i in range(len(ratio)):
    sent=ratio.iloc[i]
    name=sent['Entity']
    df=df.append({'sector':name},ignore_index=True)
    for j in range(1975,2022):
        if j in ratio.loc[ratio['Entity']==name]['Year'].to_list():
            df.loc[df['secotr']==name,j]=ratio[(ratio['Entity']==name) & (ratio['Year']==j)]['Elderly literacy rate, population 65+ years, both sexes (%)'].values[0].round(2)
        else:
            df.loc[df['secotr']==name,j]=np.nan

,Code,Year,"Elderly literacy rate, population 65+ years, both sexes (%)","Youth literacy rate, population 15-24 years, both sexes (%)",Population (historical estimates),Continent
Entity,,,,,,
Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
Afghanistan,AFG,1979,6.91031,30.066351,12986378.0,NaN
Afghanistan,AFG,2011,20.32185,46.990050,29249156.0,NaN
Afghanistan,AFG,-10000,NaN,NaN,14737.0,NaN
Afghanistan,AFG,-9000,NaN,NaN,20405.0,NaN
...,...,...,...,...,...,...
Zimbabwe,ZWE,2017,NaN,NaN,14751101.0,NaN
Zimbabwe,ZWE,2018,NaN,NaN,15052191.0,NaN
Zimbabwe,ZWE,2019,NaN,NaN,15354606.0,NaN
